In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from datetime import datetime

from ast import literal_eval

from scipy.cluster import hierarchy
from scipy.spatial import distance

from sklearn.base import clone
from sklearn.preprocessing import StandardScaler,Normalizer
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score,mean_absolute_error

import glob, os 


In [2]:
def make_walkforward_model(features,outcome,algo=LinearRegression()):
    recalc_dates = features.resample('Y').mean().index.values[:-1]
    
    ## Train models
    models = pd.Series(index=recalc_dates)
    for date in recalc_dates:    
        X_train = features.loc[slice(None,date)]
        y_train = outcome.loc[slice(None,date)]
#         print(f'Train with data prior to: {date} ({y_train.count()} obs)')
        
        model = clone(algo)
        model.fit(X_train,y_train)
        models.loc[date] = model

    begin_dates = models.index
    end_dates = models.index[1:].append(pd.to_datetime(['2099-12-31']))

    ## Generate OUT OF SAMPLE walk-forward predictions
    predictions = pd.Series(index=features.index)
    for i,model in enumerate(models): #loop thru each models object in collection
#         print(f'Using model trained on {begin_dates[i]}, Predict from: {begin_dates[i]} to: {end_dates[i]}')
        X = features.loc[slice(begin_dates[i],end_dates[i])]
        p = pd.Series(model.predict(X),index=X.index)
        predictions.loc[X.index] = p
    
    return models,predictions

def calc_scorecard(y_pred,y_true):
    
    def make_df(y_pred,y_true):
        y_pred.name = 'y_pred'
        y_true.name = 'y_true'

        df = pd.concat([y_pred,y_true],axis=1).dropna()

        df['sign_pred'] = df.y_pred.apply(np.sign)
        df['sign_true'] = df.y_true.apply(np.sign)
        df['is_correct'] = 0
        df.loc[df.sign_pred * df.sign_true > 0 ,'is_correct'] = 1 # only registers 1 when prediction was made AND it was correct
        df['is_incorrect'] = 0
        df.loc[df.sign_pred * df.sign_true < 0,'is_incorrect'] = 1 # only registers 1 when prediction was made AND it was wrong
        df['is_predicted'] = df.is_correct + df.is_incorrect
        df['result'] = df.sign_pred * df.y_true 
        return df
    
    df = make_df(y_pred,y_true)
    
    scorecard = pd.Series()
    
    # building block metrics
    scorecard.loc['RSQ'] = r2_score(df.y_true,df.y_pred)
    scorecard.loc['MAE'] = mean_absolute_error(df.y_true,df.y_pred)
    scorecard.loc['directional_accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()
    # derived metrics
    
    scorecard.loc['edge_long'] = df[df.sign_pred == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_short'] = df[df.sign_pred == -1].result.mean()  - df.y_true.mean()

    scorecard.loc['edge_win'] = df[df.is_correct == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_lose'] = df[df.is_incorrect == 1].result.mean()  - df.y_true.mean()

    
    scorecard.loc['edge_to_noise'] = scorecard.loc['edge'] / scorecard.loc['noise']
    scorecard.loc['edge_to_mae'] = scorecard.loc['edge'] / scorecard.loc['MAE']

    
    return scorecard    


def prepare_Xy(X_raw,y_raw):
    ''' Utility function to drop any samples without both valid X and y values'''
    Xy = X_raw.join(y_raw).replace({np.inf:None,-np.inf:None}).dropna()
    X = Xy.iloc[:,:-1]
    y = Xy.iloc[:,-1]
    return X,y

In [3]:
result = pd.DataFrame()
path = os.getcwd() + "/stock"
ls = glob.glob(os.path.join(path, "*.csv"))


targets = ['fut_lag1_return_1', 'fut_lag1_return_5', 'fut_lag1_return_10', 'fut_lag1_return_20',
           'fut_lag2_return_1', 'fut_lag2_return_5', 'fut_lag2_return_10', 'fut_lag2_return_20']
# targets = ['fut_lag1_return_10', 'fut_lag1_return_20',
#            'fut_lag2_return_10', 'fut_lag2_return_20']

for target in targets:
    for i in ls:
        stockCode = i[-12:-4]
        print(stockCode)
        
        try:
            stock1 = pd.read_csv(i)
            stock1 = stock1[['trade_date','close','vol']]
            stock1['trade_date'] = stock1['trade_date'].astype('str').apply(lambda x: datetime.strptime(x, "%Y%m%d"))
            stock1 = stock1.set_index('trade_date').sort_index().fillna(method='ffill')

            ### Price Feature Matrix

            # pct change
            stock1['past_ret_1'] = stock1.close.pct_change(1) # past day's returns
            stock1['past_ret_5'] = stock1.close.pct_change(5) # past week's returns
            stock1['past_ret_10'] = stock1.close.pct_change(10) # past two weeks' returns
            stock1['past_ret_20'] = stock1.close.pct_change(20) # past month's (approx) returns
            stock1['logVol'] = stock1.vol.apply(np.log)
            stock1['past_logVol_1'] = stock1.logVol.pct_change(1) # past day's change of log volume
            stock1['past_logVol_5'] = stock1.logVol.pct_change(5) # past week's change of log volume
            stock1['past_logVol_10'] = stock1.logVol.pct_change(10) # past two weeks' change of log volume
            stock1['past_logVol_20'] = stock1.logVol.pct_change(20) # past month's change of log volume

            # rolling mean or std
            stock1['roll_Close_Mean_200'] = stock1['close'].rolling(window=200, min_periods=20).mean().shift(1)
            stock1['roll_Close_Std_200'] = stock1['close'].rolling(window=200, min_periods=20).std().shift(1)
            stock1['roll_LogVol_Mean_200'] = stock1['logVol'].rolling(window=200, min_periods=20).mean().shift(1)
            stock1['roll_LogVol_Std_200'] = stock1['logVol'].rolling(window=200, min_periods=20).std().shift(1)

            # z-score
            stock1['zscore_price'] = (stock1['close'] - stock1['roll_Close_Mean_200'] ) / stock1['roll_Close_Std_200'] 
            stock1['zscore_logVol'] = (stock1['logVol'] - stock1['roll_LogVol_Mean_200'] ) / stock1['roll_LogVol_Std_200'] 

            # change sign
            stock1['sign_LogVol_1'] = stock1['past_logVol_1'].apply(np.sign)
            stock1['sign_Ret_1'] = stock1['past_ret_1'].apply(np.sign)

            ### Outcome Feature

            out_stock1 = pd.DataFrame(index = stock1.index)


            ### Assume T+1 enter the market
            out_stock1['fut_lag1_return_1'] = stock1.close.pct_change(1).shift(-(1+1)) # next day's returns
            out_stock1['fut_lag1_return_5'] = stock1.close.pct_change(5).shift(-(1+5)) # next week's returns
            out_stock1['fut_lag1_return_10'] = stock1.close.pct_change(10).shift(-(1+10)) # next two weeks' returns
            out_stock1['fut_lag1_return_20'] = stock1.close.pct_change(20).shift(-(1+20)) # next month's (approx) returns
            ### Assume T+2 enter the market
            out_stock1['fut_lag2_return_1'] = stock1.close.pct_change(1).shift(-(2+1)) # next day's returns
            out_stock1['fut_lag2_return_5'] = stock1.close.pct_change(5).shift(-(2+5)) # next week's returns
            out_stock1['fut_lag2_return_10'] = stock1.close.pct_change(10).shift(-(2+10)) # next two weeks' returns
            out_stock1['fut_lag2_return_20'] = stock1.close.pct_change(20).shift(-(2+20)) # next month's (approx) returns

            #   dropna
            out_stock1 = out_stock1.dropna()

            ### News Feature Matrix

            news = pd.read_csv('dummy_matrix.csv').drop('Unnamed: 0', axis =1)
            news['date'] =  pd.to_datetime(news['date'], format='%Y-%m-%d')

            # The keywords of news
            news_keywords = pd.read_csv('news_sent_kw_tag.csv')[['date', 'keywords']]

            # The keywords of the companys
            stock_keywords = pd.read_csv('stock_keywords.csv')
            stock_keywords['keywords'] = stock_keywords['keywords'].apply(literal_eval)
            keyword1 = stock_keywords[stock_keywords['code'] == stockCode ]['keywords'].iloc[0]

            # Find those relevant news for this compnay

            news_keywords['relevant'] = news_keywords['keywords'].apply(lambda x: any([word in x for word in keyword1]))

            ### Relevant News
            news = news.iloc[news_keywords[news_keywords['relevant']].index]
            news['date'] = pd.to_datetime(news['date'], format='%Y-%m-%d')

            ### Summarise the News on that date
            groupbyNews = news.groupby('date').agg({'lv1_tag_国际': 'sum', 'lv1_tag_时事': 'sum',
                                                    'lv1_tag_社会': 'sum', 'lv1_tag_财经': 'sum',
                                                    'sentiment_pos': 'mean',
                                                    'dayofweek_1': 'mean', 'dayofweek_2': 'mean', 'dayofweek_3': 'mean',
                                                    'dayofweek_4': 'mean', 'dayofweek_5': 'mean', 'dayofweek_6': 'mean',
                                                    'quarter_2': 'mean', 'quarter_3': 'mean', 'quarter_4': 'mean',
                                                    'month_2': 'mean', 'month_3': 'mean', 'month_4': 'mean', 'month_5': 'mean',
                                                    'month_6': 'mean', 'month_7': 'mean', 'month_8': 'mean', 'month_9': 'mean',
                                                    'month_10': 'mean', 'month_11': 'mean','month_12': 'mean',})

            # The preditive feature matrix
                # if the news is not released in a trading day, then ignore

            feature1 = groupbyNews.join(stock1, how = 'inner')

            ### Standardize predictive features
            # maybe problematic here, consider other schemes
#             std_scaler = StandardScaler()
            features_scaled = feature1.dropna()

            df = pd.DataFrame(features_scaled,index=feature1.dropna().index)
            df.columns = feature1.dropna().columns

            ### Find the indices that both feature matrix and the outcome matrix share
            index = df.index.intersection(out_stock1[target].index)
            out = out_stock1[target].loc[index]
            df = df.loc[index]

            ### Feature Selection
            corrThres = 0.1

            corr = df.corrwith(out)
            selected_features = corr[abs(corr)>corrThres].sort_values(ascending=False).index.tolist()


            X = df[selected_features]
            y = out

            linear_models,linear_preds = make_walkforward_model(X,y,algo=LinearRegression())
            tree_models,tree_preds = make_walkforward_model(X,y,algo=ExtraTreesRegressor())


            X_ens, y_ens = prepare_Xy(X_raw=pd.concat([linear_preds.rename('linear'),tree_preds.rename('tree')],
                                                      axis=1),y_raw=y)

            ensemble_models,ensemble_preds = make_walkforward_model(X_ens,y_ens,algo=LassoCV(positive=True))
            ensemble_preds = ensemble_preds.rename('ensemble')

            # calculate scores for each model
            score_ens = calc_scorecard(y_pred=ensemble_preds,y_true=y_ens).rename('Ensemble')
            score_linear = calc_scorecard(y_pred=linear_preds,y_true=y_ens).rename('Linear')
            score_tree = calc_scorecard(y_pred=tree_preds,y_true=y_ens).rename('Tree')

            scores = pd.concat([score_linear,score_tree,score_ens],axis=1)

            stat = scores.unstack().to_frame().sort_index(level=1).T
            stat.columns = stat.columns.map('_'.join)
            stat['sampleSize'] = len(df)
            stat['stock'] = stockCode

            result = result.append(stat, ignore_index=True)

        except:
            print("Unsuccessful: ", stockCode)
            pass
                                                                               
    result[result['sampleSize'] > 800].to_csv(target + "_result.csv")
    result = pd.DataFrame()
                                                                               
                                                                               
    

601318SH
Unsuccessful:  601318SH
600519SH


/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/

601800SH
Unsuccessful:  601800SH
601939SH
Unsuccessful:  601939SH
000333SZ
Unsuccessful:  000333SZ
601668SH


/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/

Unsuccessful:  601668SH
600000SH
Unsuccessful:  600000SH
600900SH
Unsuccessful:  600900SH
601998SH
Unsuccessful:  601998SH
601288SH


...

/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/

601186SH


/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/

Unsuccessful:  601186SH
601601SH


/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/Rex/anaconda3/lib/python3.6/site-packages/sklearn/